In [ ]:
#this notebook was run in google collab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!sudo apt install texlive texlive-latex-extra texlive-fonts-recommended dvipng
!sudo apt-get install texlive-latex-extra texlive-fonts-recommended dvipng cm-super
!pip install latex

In [ ]:
df = pd.read_csv('results/results_performance_profiles.csv', header=None)
df.columns = ['index', 'methods', 'subjects', 'folds', 'acc', 'kappa', 'inv-kappa']
df = df.set_index('index', drop = True)
df

In [ ]:
df = pd.read_csv('results_performance_profiles.csv', index_col=0, header=None)
df.columns = ['methods', 'subjects', 'folds', 'acc', 'kappa', 'inv-kappa']
df[df['methods'] == 'df_FF_64']

In [ ]:
method_dict = {
    'df_TT_32': '$EEGRCBAM_{32}$',
    'df_FF_32': '$EEGCBAM_{32}$',
    'df_TT_64': '$EEGRCBAM_{64}$',
    'df_FF_64': '$EEGCBAM_{64}$',
    'eegnet64': '$EEGNet_{64}$',
    'eegnet32': '$EEGNet_{32}$',
}

In [ ]:
def performance_profiles(
        df,
        cores = ['#f05039', '#00cc00', '#1f449caa'],
        tipos = ['-', '--'],
):
    vetor = []
    for subject in np.unique(np.array(df['subjects'])):
        for method in np.unique(np.array(df['methods'])):
            temp_df = df[(df['subjects'] == subject) & (df['methods'] == method)]
            vetor.append([method, subject, (1 / temp_df['kappa']).mean()])

    vetor_df = pd.DataFrame(vetor, columns=['methods', 'subjects', 'avg_acc'])

    subject_max = []
    for subject in np.unique(np.array(df['subjects'])):
        subject_max.append([subject, vetor_df[(vetor_df['subjects'] == subject)]['avg_acc'].min()])

    subject_max_df = pd.DataFrame(subject_max, columns=['subject', 'max'])

    taus_values = []
    for i in range(len(vetor_df)):
        tau = np.array(vetor_df.iloc[i]['avg_acc'] / subject_max_df[subject_max_df['subject'] == vetor_df.iloc[i]['subjects']]['max']).min()
        taus_values.append(tau)

    vetor_df['tau'] = taus_values

    teste = np.sort(np.array(vetor_df['tau']))
    min_tau = teste[0]
    max_tau = teste[-1]

    var1 = 0
    var2 = 0
    for method in np.unique(np.array(df['methods'])):
        teste = np.sort(np.array(vetor_df[vetor_df['methods'] == method]['tau']))
        taus = [min_tau, min_tau]
        ro = [0, 0]
        for t in teste:
            if t == taus[-1]:
                ro[-1] += 1/9
            else:
                taus.append(t)
                ro.append(ro[-1])
                taus.append(t)
                ro.append(ro[-1] + 1/9)

        taus.append(max_tau)
        ro.append(1)
        plt.plot(taus, ro, tipos[var2 % 2], label=method_dict[method], color=cores[var1 // 2])
        var1 += 1
        var2 += 1

    plt.xlim([min_tau, max_tau])
    plt.xscale('log')
    plt.xticks([1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2], [1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2])
    plt.xlabel(r'$\tau$')
    plt.ylabel(r'$\rho$($\tau$)')
    plt.legend(fontsize="10")
    # plt.savefig('performance_profile.pdf')
    # plt.show()

plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 12
plt.figure(figsize=(5, 3))


performance_profiles(df[df['methods'].isin(['df_TT_32', 'df_FF_32', 'df_TT_64', 'df_FF_64', 'eegnet64', 'eegnet32'])])
plt.savefig('performance_profile.pdf', format='pdf', bbox_inches='tight')